# 모두의 말뭉치 - 비출판물 말뭉치
> https://kli.korean.go.kr/corpus/request/corpusList.do

In [34]:
# Hugging Face Transfomers 환경 세팅
!pip install transformers torch
!pip install kobert-transformers

   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ------ --------------------------------- 1.8/10.8 MB 10.1 MB/s eta 0:00:01
   -------------- ------------------------- 3.9/10.8 MB 9.4 MB/s eta 0:00:01
   --------------------- ------------------ 5.8/10.8 MB 9.5 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.8 MB 8.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/10.8 MB 8.8 MB/s eta 0:00:01
   -------------------------------------- - 10.5/10.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 10.8/10.8 MB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ----------------------------- ---------- 1.8/2.5 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 7.6 MB/s eta 0:00:00


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 991.5/991.5 kB 9.2 MB/s eta 0:00:00


In [4]:
# 패키지 설치 및 라이브러리 불러오기
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy==1.23.1

     ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
     ---------------------------------- ----- 4.5/5.1 MB 22.4 MB/s eta 0:00:01
     ---------------------------------------- 5.1/5.1 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/33.1 MB ? eta -:--:--
   - -------------------------------------- 1.6/33.1 MB 7.6 MB/s eta 0:00:05
   ------ --------------------------------- 5.2/33.1 MB 12.3 MB/s eta 0:00:03
   ------------- -------------------------- 11.3/33.1 MB 18.0 MB/s eta 0:00:02
   -------------------- ------------------- 17.0/33.1 MB 20.3 MB/s eta 0:00:01
   --------------------------- ------------ 22.8/33.1 MB 21.5 MB/s eta 0:00:01
   ----------------------------------- ---- 29.1/33.1 MB 22.8 MB/s eta 0:00:01
   ---------------------------------------- 33.1/33.1 MB 22.1 MB/s eta 0:00:00
  Running setup.py clean for numpy
Failed to buil

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Running from numpy source directory.
      C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-h5ele26c\numpy_6cd8e299d1cf4da99e6a2566c1fd37f7\numpy\distutils\misc_util.py:476: SyntaxWarning: "is" with a literal. Did you mean "=="?
        return is_string(s) and ('*' in s or '?' is s)
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-h5ele26c\numpy_6cd8e299d1cf4da99e6a2566c1fd37f7\setup.py", line 419, in <module>
          setup_package()
        File "C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-h5ele26c\numpy_6cd8e299d1cf4da99e6a2566c1fd37f7\setup.py", line 398, in setup_package
          from numpy.distutils.core import setup
        File "C:\Users\Pu

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cython-3.1.2-cp311-cp311-win_amd64.whl.metadata (6.0 kB)
Using cached cython-3.1.2-cp311-cp311-win_amd64.whl (2.7 MB)
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp311-cp311-win_amd64.whl size=455721 sha256=a4fc8acc4375056aa04f2d90164a0d6228883c5f8891dfce37af49c71ab7df5f
  Stored in directory: c:\users\wooll\appdata\local\pip\cache\wheels\29\64\e3\047328bfc4a551696adc4a82dba3c980a1dad62198b8a3bb83
Successfully built gluonnlp


     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
     ---------- ----------------------------- 2.9/10.7 MB 15.2 MB/s eta 0:00:01
     ------------------------------ --------- 8.1/10.7 MB 19.3 MB/s eta 0:00:01
     --------------------------------------- 10.7/10.7 MB 18.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for numpy: filename=numpy-1.23.1-cp311-cp311-win_amd64.whl size=5956531 sha256=9ae227d161100d7f0bbf6d16af38391c79fe4770f09f2991d4cf7267b39a4700
  Stored in directory: c:\users\wooll\appdata\local\pip\cache\wheels\54\67\ec\c3e57b4b51328fb39dd4d63906b0d3bd37a312508e5922682f
Successfully built numpy
  Attempting uninstall: numpy
    Found exis

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ml-dtypes 0.4.1 requires numpy>=1.23.3; python_version >= "3.11", but you have numpy 1.23.1 which is incompatible.
pandas 2.2.3 requires numpy>=1.23.2; python_version == "3.11", but you have numpy 1.23.1 which is incompatible.
scipy 1.14.1 requires numpy<2.3,>=1.23.5, but you have numpy 1.23.1 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.1 which is incompatible.


In [ ]:
import re # 텍스트 전처리 용도
import pandas as pd # 데이터 구조 및 분석
from sklearn.model_selection import train_test_split # 모델 평가를 위한 데이터 분할
from sklearn.feature_extraction.text import TfidfVectorizer # 텍스트를 TF-IDF 피처 벡터로 변환
from sklearn.preprocessing import MultiLabelBinarizer # 멀티라벨 레이블 리스트를 이진 매트릭스로 변환 (다중 감정 분류용)
from sklearn.multiclass import OneVsRestClassifier # 멀티라벨/다중 클래스 분류 전략: 각 레이블마다 독립적인 이진 분류기 학습
from sklearn.linear_model import LogisticRegression  # 선형 분류 모델 
from sklearn.pipeline import Pipeline # 전처리/벡터화/분류기 단계 순차적으로 연결 (코드 간결화 위함)
from sklearn.metrics import classification_report # 분류 성능 리포트 생성 
from collections import Counter # 빈도 집계 자료구조 (토큰 빈도 계산, 후보 단어 추출 등에 사용)
from itertools import chain # 이터러블 평탄화 도구: 토큰 리스트 -> 단일 리스트 변환 시 활용 

In [13]:
# 데이터 로드 
df = pd.read_csv(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\NIKL_NP.csv')
df.head()

,Unnamed: 0,file_id,anno_level,category,title,author_id,author_age,author_occupation,author_sex,author_submission,author_handwriting,text_date,text_subclass,sentence
0,0,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어제는 아내의 생일이었다. 생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터...
1,1,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,"주된 메뉴는 스테이크와 낙지볶음, 미역국, 잡채, 소야 등이었다. 스테이크는 자주하..."
2,2,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,그런데 상상도 못한 일이 벌이지고 말았다. 보통 시즈닝이 되지 않은 원육을 사서 스...
3,3,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,앞면을 센불에 1분을 굽고 뒤집는 순간 방부제가 함께 구어진 것을 알았다. 아내의 ...
4,4,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어처구니 없는 상황이 발생한 것이다. 방부제가 센불에 녹아서 그런지 물처럼 흘러내렸...


- 데이터셋의 문제: 비출판물이다보니 띄어쓰기가 옳게 되어있지 않는 경우가 있음

In [47]:
# 결측값 확인
df.isnull().sum()

Unnamed: 0             0
file_id                0
anno_level             0
category               0
title                  0
author_id              0
author_age             0
author_occupation     27
author_sex             0
author_submission      0
author_handwriting     0
text_date              0
text_subclass          0
sentence               0
dtype: int64

In [48]:
# 개요 확인
print(f"총 문장 수: {len(df)}")
print('------------------------------------------')
print("title별 분포:")
print(df['title'].value_counts(), '\n')
print('------------------------------------------')
print("문장 길이(문자 수) 통계:")
print(df['sentence'].str.len().describe(), '\n')

총 문장 수: 76805
------------------------------------------
title별 분포:
title
아르반 일지                322
짧은 사랑의 단상             320
Rumex                 295
무제                    287
제목없음                  263
                     ... 
과학과 인문학의 간극에 대한 고찰      1
요즘 나의 생각들               1
출근하기 싫다                 1
독립적인                    1
땅속나라 도둑괴물               1
Name: count, Length: 9448, dtype: int64 

------------------------------------------
문장 길이(문자 수) 통계:
count    76805.000000
mean       114.733312
std        198.525188
min          1.000000
25%         19.000000
50%         44.000000
75%        126.000000
max       5630.000000
Name: sentence, dtype: float64 



In [49]:
# 중복되는 행 있는 지 확인
df.duplicated().sum()

0

## TF-IDF + LogisticRegression 분류기 - Baseline

In [14]:
# 한국어 불용어 리스트 로드
stopwords_kr = set()
with open(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\stopwords-ko.txt') as f:
  for line in f:
    w = line.strip()
    if w:
      stopwords_kr.add(w)    

In [ ]:
# 텍스트 정제 사용 

In [ ]:
# 정졔 + 토큰화 함수 (한글 2글자 이상 단어만 추출)

def tokenize_kr(text):
  # 한글/공백 제외 모두 스페이스로
  text = re.sub(r'[^가-힣\s]', ' ', text)
  # 다중 공백 → 단일, 양끝 공백 제거
  text = re.sub(r'\s+', ' ', text).strip()
  # 2글자 이상 한글 단어 추출
  tokens = re.findall(r'[가-힣]{2,}', text)
  # 불용어 제거
  return [t for t in tokens if t not in stopwords_kr]

In [40]:
'''
# 기본 감정 라벨 사전 정의 (라벨링 된 문장 비율 1% 도 안나왔음..)
lexicon = {
    '행복': ['행복', '기쁘', '즐거', '환희'],
    '슬픔': ['슬픔', '슬프', '우울', '비통', '눈물'],
    '분노': ['분노', '화나', '열받', '격분'],
    '공포': ['공포', '무섭', '두렵'],
    '혐오': ['혐오', '싫'],
    '놀람': ['놀라', '충격']
}
'''

"\n# 기본 감정 라벨 사전 정의 (라벨링 된 문장 비율 1% 도 안나왔음..)\nlexicon = {\n    '행복': ['행복', '기쁘', '즐거', '환희'],\n    '슬픔': ['슬픔', '슬프', '우울', '비통', '눈물'],\n    '분노': ['분노', '화나', '열받', '격분'],\n    '공포': ['공포', '무섭', '두렵'],\n    '혐오': ['혐오', '싫'],\n    '놀람': ['놀라', '충격']\n}\n"

In [ ]:
# 감정 Label (인사이드 아웃 2 기반)
# 기본 감정 감정 사전으로 했을 때 보다 라벨링 된 문장 비율 올라감. 
lexicon={
  '기쁨': ['행복', '기쁘', '즐거', '환희', '기쁨'], # 기쁨이 (Joy)
  '슬픔': ['슬픔', '슬프', '우울', '비통', '눈물', '상심'], # 슬픔이 (Sadness)
  '버럭': ['분노', '화나', '열받', '격분', '버럭', '분개', '화가'], # 버럭이 (Anger)
  '까칠': ['혐오', '싫', '역겹', '불쾌', '까칠', '거북', '싫증'], # 까칠이 (Disgust)
  '소심': ['무섭', '두렵', '겁', '소심', '겁나다', '겁먹'], # 소심이 (Fear)
  '불안': ['불안', '초조', '긴장', '떨리', '걱정', '안절부절'], # 불안이 (Anxiety)
  '부러움': ['부럽', '부러움', '질투', '질투심', '시기'], # 부럽이 (Envy)
  '당황': ['당황', '부끄러움', '민망', '어색', '당황스러움'], # 당황이 (Embarrassment)
  '따분': ['따분', '지루', '귀찮', '권태', '심심', '지루함'] # 따분이 (Ennui)
}

In [17]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from collections import Counter
from itertools import chain

In [18]:

# 룰 기반 멀티라벨 추출 함수
def extract_emotions_kr(tokens):
    found = set()
    # lexicon: 레이블(문자열) 해당 감정 어근 리스트
    for label, keywords in lexicon.items():
        # 토큰(token) 하나하나에 대해 키워드가 포함되어 있는지 확인
        for tok in tokens:
            for kw in keywords:
                if kw in tok:
                    found.add(label)
                    # 이미 label을 찾았으므로 다음 레이블로 넘어감
                    break
    return list(found)

In [19]:
# 전처리 + 멀티라벨 컬럼 생성
df['tokens']    = df['sentence'].apply(tokenize_kr)
df['emotions']  = df['tokens'].apply(extract_emotions_kr)

In [20]:
# 룰 기반 커버리지 확인
df['has_emotion'] = df['emotions'].apply(lambda lst: bool(lst))
print("라벨링된 문장 비율:", df['has_emotion'].mean())

라벨링된 문장 비율: 0.14301152268732503


In [23]:
'''
# 아직 라벨링 안 된 문장들의 토큰 모으기
unlab_tokens = df.loc[~df['has_emotion'], 'tokens'].tolist()
all_tokens   = list(chain.from_iterable(unlab_tokens))

# 빈도 계산
freq = Counter(all_tokens)

# 상위 500개 혹은 전체 후보를 DataFrame으로
cand_df = pd.DataFrame(freq.most_common(), columns=['token','count'])

# 엑셀로 저장 → 사람이 옆에 emotion 컬럼 달아서 검수
cand_df.to_excel('emotion_token_candidates.xlsx', index=False)
'''

"\n# 아직 라벨링 안 된 문장들의 토큰 모으기\nunlab_tokens = df.loc[~df['has_emotion'], 'tokens'].tolist()\nall_tokens   = list(chain.from_iterable(unlab_tokens))\n\n# 빈도 계산\nfreq = Counter(all_tokens)\n\n# 상위 500개 혹은 전체 후보를 DataFrame으로\ncand_df = pd.DataFrame(freq.most_common(), columns=['token','count'])\n\n# 엑셀로 저장 → 사람이 옆에 emotion 컬럼 달아서 검수\ncand_df.to_excel('emotion_token_candidates.xlsx', index=False)\n"

In [32]:
# 라벨이 있는 문장만 추출
df_lbl = df.dropna(subset=['emotions'])
X, y = df_lbl['sentence'], df_lbl['emotions']

In [35]:
# MultiLabelBinarizer 로 y 변환
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df_lbl['emotions'])  

In [ ]:
# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    df_lbl['sentence'], Y,
    test_size=0.2,
    random_state=42
)

In [37]:
# 파이프라인 정의 (TF-IDF + 로지스틱 회귀)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        tokenizer=tokenize_kr,   # 앞서 정의한 한글 토크나이저
        preprocessor=lambda x: x,
        token_pattern=None,
        ngram_range=(1,2),
        min_df=3
    )),
    ('clf', OneVsRestClassifier(
        LogisticRegression(class_weight='balanced', max_iter=1000),
        n_jobs=-1
    ))
])

In [38]:
# 학습
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(min_df=3, ngram_range=(1, 2),
                                 preprocessor=<function <lambda> at 0x000002D9F6916C00>,
                                 token_pattern=None,
                                 tokenizer=<function tokenize_kr at 0x000002D9E05A8400>)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                                  max_iter=1000),
                                     n_jobs=-1))])

In [39]:
# 예측 및 평가 
y_pred = pipeline.predict(X_test)
print("\n=== 분류기 성능 ===")
print(classification_report(y_test, y_pred, digits=4))


=== 분류기 성능 ===
              precision    recall  f1-score   support

           0     0.5407    0.7423    0.6256       582
           1     0.4061    0.6086    0.4871       419
           2     0.3576    0.4574    0.4014       129
           3     0.2684    0.4195    0.3274       174
           4     0.4646    0.6580    0.5446       269
           5     0.2636    0.3987    0.3174       158
           6     0.3603    0.6266    0.4575       391
           7     0.4005    0.6135    0.4846       282
           8     0.5216    0.6847    0.5921       406

   micro avg     0.4250    0.6246    0.5058      2810
   macro avg     0.3981    0.5788    0.4709      2810
weighted avg     0.4305    0.6246    0.5087      2810
 samples avg     0.0771    0.0880    0.0783      2810



C:\Users\wooll\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\wooll\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\wooll\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# 성능 개선

## 

## 1. Lexicon 확장 및 전처리 고도화
> https://velog.io/@ganta/%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%A0%84%EC%B2%98%EB%A6%AC-%ED%8C%A8%ED%82%A4%EC%A7%80Text-Preprocessing-Tools-for-Korean-Text

In [ ]:
# 네이버 맞춤법 검사기 패키지 설치 
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to c:\users\public\documents\estsoft\creatortemp\pip-req-build-rg_cqoh5
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4894 sha256=9a5a15b7a509c6763737f2c55b65ccb71f001b2e92da566846919c9dcf040e1e
  Stored in directory: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-ephem-wheel-cache-tr791zoz\wheels\18\0f\39\d0e3972de3368ba3ab62817cd7b17a74123414afd774ce604b
Successfully built py-hanspell


  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-rg_cqoh5'


In [ ]:
# Numpy 버전과 TensorFlow 가 요구하는 ABI 맞지 않음
# 이를 위해 호환되는 Numpy 버전(1.23.5)으로 재설치
!pip install --upgrade --force-reinstall numpy==1.23.5

  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.


In [68]:
# Numpy 버전 맞춘 후 TensorFlow 재설치 -> 왜이렇게 오래 걸리지
!pip install --upgrade --force-reinstall numpy==1.23.5
!pip install tensorflow==2.12.0

  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.


INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/272.9 MB ? eta -:--:--
    --------------------------------------- 3.9/272.9 MB 21.3 MB/s eta 0:00:13
   - -------------------------------------- 10.0/272.9 MB 24.9 MB/s eta 0:00:11
   -- ------------------------------------- 15.5/272.9 MB 25.6 MB/s eta 0:00:11
   --- ------------------------------------ 22.0/272.9 MB 27.3 MB/s eta 0:00:10
   ---- ----------------------------------- 28.0/272.9 MB 27.4 MB/s eta 0:00:09
  

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pykospacing 0.5 requires tensorflow>=2.16.2, but you have tensorflow 2.12.0 which is incompatible.


In [69]:
# 한국어 띄어쓰기 패키지 설치
# pykospacing 재설치
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\public\documents\estsoft\creatortemp\pip-req-build-9mj77z7a
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
    --------------------------------------- 5.0/375.9 MB 21.5 MB/s eta 0:00:18
   - -------------------------------------- 10.0/375.9 MB 23.0 MB/s eta 0:00:16
   - -------------------------------------- 13.6/375.9 MB 19.5 MB/s eta 0:00:19
   - -------------------------------------- 17.8/375.9 MB 19.7 MB/s eta 0:00:19
  

  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-9mj77z7a'
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 3.10.0 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.19.0 which is incompatible.


In [71]:
!pip install --upgrade --force-reinstall \
  numpy==1.23.5 \
  keras==2.12.0 \
  tensorboard==2.12.0 \
  tensorflow==2.12.0

  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached tensorflow-2.12.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached tensorflow_intel-2.12.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached jax-0.6.2-py3-none-any.whl.metadata (13 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached

error: uninstall-no-record-file

× Cannot uninstall wheel None
╰─> The package's contents are unknown: no RECORD file was found for wheel.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps wheel==0.44.0


In [ ]:
# 설치 후 콘솔 재시작
# 버전 제대로 맞춰졌는지 확인
import numpy as np, tensorflow as tf, keras, tensorboard
print(np.__version__, tf.__version__, keras.__version__, tensorboard.__version__)

In [70]:
from pykospacing import Spacing
from hanspell import spell_checker

TypeError: Couldn't build proto file into descriptor pool: duplicate symbol 'tensorflow.CoordinatedJob'

In [ ]:
# 데이터 로드
df = pd.read_csv(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\NIKL_NP.csv')
df.head()

In [54]:
def correct_spelling(text):
    """
    네이버 맞춤법 검사기(hanspell)를 사용해 맞춤법을 교정합니다.
    API 오류나 예상치 못한 응답 구조 변경 시 예외를 잡아
    원문(text)을 그대로 반환하도록 합니다.
    """
    try:
        # 맞춤법 검사 실행
        checked = spell_checker.check(text)
        # checked.checked 에 교정된 문자열이 들어 있습니다.
        return checked.checked
    except KeyError as e:
        # 응답에 'result' 키가 없어서 KeyError 발생 시
        print(f"[warning] hanspell KeyError: {e}. 원문을 반환합니다.")
        return text
    except Exception as e:
        # 그 외 네트워크 오류, JSON 파싱 오류 등 모든 예외를 포착
        print(f"[warning] hanspell failed ({type(e).__name__}): {e}. 원문을 반환합니다.")
        return text

# 적용 예
raw = "오늘 날씨가 넘 좋아요~"
print(correct_spelling(raw))  # → "오늘 날씨가 너무 좋아요~"

[warning] hanspell KeyError: 'result'. 원문을 반환합니다.
오늘 날씨가 넘 좋아요~


In [55]:
# 띄어쓰기 교정기
spacing = Spacing()
def correct_spacing(text):
    try:
        return spacing(text)
    except Exception:
        return text

# 맞춤법 교정기 with 예외 처리
def correct_spelling(text):
    try:
        checked = spell_checker.check(text)
        return checked.checked
    except Exception:
        return text

# 특수문자·반복문자 정제
def clean_punctuation(text):
    # 연속된 .,!? 제거
    text = re.sub(r'([!?\.])\1+', r'\1', text)
    # 불필요 기호 제거
    text = re.sub(r'[※☆★♡♪]', ' ', text)
    # 다중 공백 정리
    return re.sub(r'\s+', ' ', text).strip()

# 통합 정규화 함수
def normalize_text(text):
    text = str(text)
    text = correct_spacing(text)         # 1) 띄어쓰기
    text = correct_spelling(text)        # 2) 맞춤법
    text = clean_punctuation(text)       # 3) 특수문자
    return text

# 기존 한글 토크나이저
def tokenize_kr(text):
    text = re.sub(r'[^가-힣\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = re.findall(r'[가-힣]{2,}', text)
    return [t for t in tokens if t not in stopwords_kr]

NameError: name 'Spacing' is not defined

In [ ]:
# 데이터 로드
df = pd.read_csv(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\NIKL_NP.csv')
df['sentence'] = df['sentence'].astype(str)

In [ ]:
# 원본 문장에 정규화 적용
df['normalized'] = df['sentence'].apply(normalize_text)

In [ ]:
# 정규화된 문장에 토큰화 적용
df['tokens'] = df['normalized'].apply(tokenize_kr)

In [ ]:
# 결과 확인
print(df[['sentence','normalized','tokens']].head())

In [42]:
# df['tokens'] -> 한글 토큰 리스트 준비되어있는 상태

# 빈도 기반 감정 어근 후보 뽑기
unlab_tokens = df.loc[~df['has_emotion'], 'tokens'].tolist() # 라벨 없는 문장들의 토크
all_unlab = list(chain.from_iterable(unlab_tokens))
freq = Counter(all_unlab)

In [47]:
# 상위 1000개 토큰 후보로 진행 -> 살펴 본 결과 감정을 나타내는 토큰이 많이 없음
# cand_freq = pd.DataFrame(freq.most_common(1000), columns=['token','count'])
# cand_freq.to_csv('candidates_freq2.csv', index=False, encoding='utf-8-sig')

# 하위 1000개 토큰 후보로 진행 -> most_common() 뒤집기
bottom500 = freq.most_common()[:-1001:-1]
cand_least = pd.DataFrame(bottom500, columns=['token','count'])
cand_least.to_csv('candidates_least_freq.csv', index=False)

# 엑셀로 열어서 직접 감정 달아줌 

In [ ]:
# 감정 Label (인사이드 아웃 2 기반)
# 기본 감정 감정 사전으로 했을 때 보다 라벨링 된 문장 비율 올라감. 
lexicon={
  '기쁨': ['행복', '기쁘', '즐거', '환희', '기쁨'], # 기쁨이 (Joy)
  '슬픔': ['슬픔', '슬프', '우울', '비통', '눈물', '상심'], # 슬픔이 (Sadness)
  '버럭': ['분노', '화나', '열받', '격분', '버럭', '분개', '화가'], # 버럭이 (Anger)
  '까칠': ['혐오', '싫', '역겹', '불쾌', '까칠', '거북', '싫증'], # 까칠이 (Disgust)
  '소심': ['무섭', '두렵', '겁', '소심', '겁나다', '겁먹'], # 소심이 (Fear)
  '불안': ['불안', '초조', '긴장', '떨리', '걱정', '안절부절'], # 불안이 (Anxiety)
  '부러움': ['부럽', '부러움', '질투', '질투심', '시기'], # 부럽이 (Envy)
  '당황': ['당황', '부끄러움', '민망', '어색', '당황스러움'], # 당황이 (Embarrassment)
  '따분': ['따분', '지루', '귀찮', '권태', '심심', '지루함'] # 따분이 (Ennui)
}

## 2. 문맥 반영 모델 도입